<a href="https://colab.research.google.com/github/umslengineering/EE1108/blob/main/EE1108_chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#chess.py

In [1]:
# chess.py
# Full chess game with rules, timer, winner, and piece sprites

BOARD_SIZE = 8
SQUARE = 80

WIDTH = BOARD_SIZE * SQUARE
HEIGHT = BOARD_SIZE * SQUARE + 60

LIGHT = (240, 217, 181)
DARK = (181, 136, 99)

turn = "white"
selected = None

white_time = 300  # 5 minutes
black_time = 300

game_over = False
winner = ""

# -----------------------------------
# Board setup
# -----------------------------------
board = [
    ["br","bn","bb","bq","bk","bb","bn","br"],
    ["bp"]*8,
    [".."]*8,
    [".."]*8,
    [".."]*8,
    [".."]*8,
    ["wp"]*8,
    ["wr","wn","wb","wq","wk","wb","wn","wr"]
]

# Load piece images
piece_images = {}
for color in ["w","b"]:
    for ptype in ["p","r","n","b","q","k"]:
        key = color+ptype
        piece_images[key] = key

# -----------------------------------
# Draw board
# -----------------------------------

def draw_row(row_index, c1, c2):
    y = row_index * SQUARE
    for col in range(BOARD_SIZE):
        x = col * SQUARE
        color = c1 if col % 2 == 0 else c2
        rect = Rect(x, y, SQUARE, SQUARE)
        screen.draw.filled_rect(rect, color)

def draw_board():
    for row in range(BOARD_SIZE):
        if row % 2 == 0:
            draw_row(row, LIGHT, DARK)
        else:
            draw_row(row, DARK, LIGHT)

# -----------------------------------
# Draw pieces
# -----------------------------------

def draw_pieces():
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            piece = board[r][c]
            if piece != "..":
                screen.blit(piece, (c*SQUARE, r*SQUARE))

# -----------------------------------
# Draw selection
# -----------------------------------

def draw_selection():
    if selected:
        r, c = selected
        rect = Rect(c*SQUARE, r*SQUARE, SQUARE, SQUARE)
        screen.draw.rect(rect, (0,255,0))

# -----------------------------------
# Chess Logic
# -----------------------------------

def path_clear(sr, sc, tr, tc):
    dr = tr - sr
    dc = tc - sc
    steps = max(abs(dr), abs(dc))
    step_r = (dr // steps) if dr != 0 else 0
    step_c = (dc // steps) if dc != 0 else 0

    for i in range(1, steps):
        r = sr + i*step_r
        c = sc + i*step_c
        if board[r][c] != "..":
            return False
    return True

def can_piece_reach(sr, sc, tr, tc):
    piece = board[sr][sc]
    if piece == "..":
        return False

    p = piece[1]
    dr = tr - sr
    dc = tc - sc

    if p == "p":
        color = piece[0]
        direction = -1 if color == "w" else 1
        return abs(dc) == 1 and dr == direction

    if p == "r":
        return (sr == tr or sc == tc) and path_clear(sr, sc, tr, tc)

    if p == "b":
        return abs(dr) == abs(dc) and path_clear(sr, sc, tr, tc)

    if p == "q":
        return (sr == tr or sc == tc or abs(dr) == abs(dc)) and path_clear(sr, sc, tr, tc)

    if p == "n":
        return (abs(dr), abs(dc)) in [(2,1),(1,2)]

    if p == "k":
        return max(abs(dr), abs(dc)) == 1

    return False

def is_square_attacked(row, col, by_color):
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            piece = board[r][c]
            if piece != ".." and piece[0] == by_color:
                if can_piece_reach(r,c,row,col):
                    return True
    return False

def find_king(color):
    king_piece = ("w" if color=="white" else "b") + "k"
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board[r][c] == king_piece:
                return (r,c)
    return None

def is_in_check(color):
    king_pos = find_king(color)
    if king_pos is None:
        return False
    opponent = "b" if color == "white" else "w"
    return is_square_attacked(king_pos[0], king_pos[1], opponent)

def would_be_in_check(sr, sc, tr, tc, color):
    original_piece = board[tr][tc]
    board[tr][tc] = board[sr][sc]
    board[sr][sc] = ".."

    in_check = is_in_check(color)

    board[sr][sc] = board[tr][tc]
    board[tr][tc] = original_piece

    return in_check

def valid_move(sr, sc, tr, tc):
    piece = board[sr][sc]
    target = board[tr][tc]

    if piece == "..":
        return False

    color = piece[0]
    p = piece[1]

    if target != ".." and target[0] == color:
        return False

    if target != ".." and target[1] == "k":
        return False

    dr = tr - sr
    dc = tc - sc
    is_legal = False

    if p == "p":
        direction = -1 if color=="w" else 1
        start_row = 6 if color=="w" else 1

        if dc==0 and dr==direction and target=="..":
            is_legal=True
        elif dc==0 and sr==start_row and dr==2*direction and board[sr+direction][sc]==".." and target=="..":
            is_legal=True
        elif abs(dc)==1 and dr==direction and target!="..":
            is_legal=True

    elif p == "r":
        if (sr==tr or sc==tc) and path_clear(sr,sc,tr,tc):
            is_legal=True

    elif p == "b":
        if abs(dr)==abs(dc) and path_clear(sr,sc,tr,tc):
            is_legal=True

    elif p == "q":
        if (sr==tr or sc==tc or abs(dr)==abs(dc)) and path_clear(sr,sc,tr,tc):
            is_legal=True

    elif p == "n":
        if (abs(dr),abs(dc)) in [(2,1),(1,2)]:
            is_legal=True

    elif p == "k":
        opponent = "b" if color=="w" else "w"
        if max(abs(dr),abs(dc))==1 and not is_square_attacked(tr,tc,opponent):
            is_legal=True

    if not is_legal:
        return False

    current_color = "white" if color=="w" else "black"
    if would_be_in_check(sr,sc,tr,tc,current_color):
        return False

    return True

def has_legal_moves(color):
    for sr in range(BOARD_SIZE):
        for sc in range(BOARD_SIZE):
            piece = board[sr][sc]
            if piece!=".." and ((color=="white" and piece[0]=="w") or (color=="black" and piece[0]=="b")):
                for tr in range(BOARD_SIZE):
                    for tc in range(BOARD_SIZE):
                        if valid_move(sr,sc,tr,tc):
                            return True
    return False

# -----------------------------------
# Mouse input
# -----------------------------------

def on_mouse_down(pos):
    global selected, turn, game_over, winner

    if game_over:
        return

    c = int(pos[0]//SQUARE)
    r = int(pos[1]//SQUARE)

    if selected is None:
        piece = board[r][c]
        if piece!=".." and ((turn=="white" and piece[0]=="w") or (turn=="black" and piece[0]=="b")):
            selected=(r,c)
    else:
        sr,sc = selected
        if valid_move(sr,sc,r,c):

            # Move piece
            board[r][c] = board[sr][sc]
            board[sr][sc] = ".."

            # ✅ Pawn Promotion
            moved_piece = board[r][c]
            if moved_piece == "wp" and r == 0:
                board[r][c] = "wq"
            elif moved_piece == "bp" and r == 7:
                board[r][c] = "bq"

            turn = "black" if turn=="white" else "white"

            if not has_legal_moves(turn):
                game_over=True
                if is_in_check(turn):
                    winner = "black" if turn=="white" else "white"
                else:
                    winner="draw"

        selected=None

# -----------------------------------
# Timer update
# -----------------------------------

def update():
    global white_time, black_time, game_over, winner

    if game_over:
        return

    if turn=="white":
        white_time-=1/60
        if white_time<=0:
            winner="black"
            game_over=True
    else:
        black_time-=1/60
        if black_time<=0:
            winner="white"
            game_over=True

# -----------------------------------
# Draw everything
# -----------------------------------

def draw():
    screen.clear()
    draw_board()
    draw_selection()
    draw_pieces()

    screen.draw.text(f"Turn: {turn}", (10, HEIGHT-55), fontsize=30, color="white")
    screen.draw.text(f"W:{int(white_time)}  B:{int(black_time)}", (200, HEIGHT-55), fontsize=30, color="white")

    if not game_over and is_in_check(turn):
        screen.draw.text("CHECK!", (WIDTH-150, HEIGHT-55), fontsize=35, color="red")

    if game_over:
        if winner=="draw":
            screen.draw.text("STALEMATE!", center=(WIDTH/2, HEIGHT-30), fontsize=40, color="yellow")
        else:
            screen.draw.text(f"{winner.upper()} WINS!", center=(WIDTH/2, HEIGHT-30), fontsize=40, color="yellow")